# Improve ChatGPT with Knowledge Graphs


In [1]:
# pip install -q openai langchain

In [2]:
# pip install python-dotenv

In [2]:
import os
import openai
from string import Template
import json
from neo4j import GraphDatabase
import glob
from timeit import default_timer as timer
from dotenv import load_dotenv
from time import sleep
from langchain.text_splitter import TokenTextSplitter
from langchain.schema import Document


In [3]:
load_dotenv()

True

In [4]:
import os
import openai

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_KEY")
openai.api_key = os.getenv("api_key_azure")
openai.api_version = os.getenv("api_version")
openai_deployment = "sdgi-gpt-35-turbo-16k" 
# print(os.getenv("api_key_azure"))
# print("=====")
# print(os.getenv("api_version"))
# completion = openai.ChatCompletion.create(model="gpt-3.5-turbo",
#                                           temperature=0,
#                                           messages=[{"role": "user",
#                                                      "content": question}])
# print(completion["choices"][0]["message"]["content"])

# completion = openai.chat.completions.create(
#                     model=openai_deployment,
#                     max_tokens=15000,
#                     temperature=0,
#                     messages=[
#                         {"role": "user", "content": question}
#                     ]
#                 )
# nlp_results = completion.choices[0].message.content
# print(nlp_results)

In [12]:
from langchain.llms import OpenAI
from langchain.indexes import GraphIndexCreator
from langchain.chains import GraphQAChain
from langchain.prompts import PromptTemplate

text = f"""
0. The country programme seeks to achieve transformational change in an 
environment of great complexity and insecurity. UNDP will continue to adapt to a 
changing national context in four of the five United Nations Development 
Assistance Framework outcomes outlined below, and will take on leadership or 
facilitation roles in those areas as requested. Conflict sensitivity will be applied 
throughout the programme and will be mainstreamed in all projects. Political, 
economic, and conflict analysis of the provinces where UNDP is working will be 
undertaken through its regional hubs at the subnational level. This will be 
accompanied by greater use of national systems, a focus on capacity development 
instead of capacity substitution, and assistance with phasing out parallel structures 
and strengthening approaches to regionalization. For each country programme 
outcome, UNDP will prepare an outcome strategy document, in consultation with all 
partners and the new Government and in the light of the evolving UNAMA mandate 
after 2015.11. UNDP will ensure that four cross-cutting parameters or guiding principles are 
addressed in the outcome strategies of the country programme to be prepared after 
the document has been approved. They are: 
(a) Area-based approaches for better targeting of beneficiaries. 
Socio-economic and baseline analysis in pilot regions and provinces will be carried 
out to ensure that direct benefits accrue to policymakers, the poor, women, and other 
selected beneficiaries, such as internally displaced persons. Participatory planning 
will foster community ownership, leading to sustainable interventions through 
conflict-sensitive programming, the rights-based approach and ‘do no harm’ 
principles.
(b) Scalability of results and use of multidisciplinary approaches. Using an 
issues-based approach, UNDP will ensure that synergies between outcomes and 
fields of work are established to ensure accelerated progress towards specified 
goals, fostering development impact and social cohesion.
(c) Partnership building. This will involve a greater use of state and 
non-state actors in Afghanistan, and at the regional and global levels through SouthSouth cooperation, to ensure that the best national and international comparative 
experience can be integrated into the country programme outcome areas of work.
(d) Use of national systems and ‘Serving as One’. Serving as One 
approaches incorporating the use of integrated work plans and joint programmes, 
coordinated through outcome-specific inter-agency working groups, have been 
agreed to.
12. Outcome 1 of the country programme document, on accountable governance, 
addresses governance deficits and responds to the need to strengthen the 
accountability of institutions and promote civil service reform, with special 
attention to selected regions and provinces. UNDP will seek to promote:
(a) Inclusive political processes and representative institutions. In 
coordination with UNAMA, UNDP electoral cycle support will assist national 
institutions in the better management of presidential, parliamentary and local 
election processes. UNDP is already a partner of the key electoral bodies. UNDP 
will support the National Assembly in building capacities to deliver on its 
constitutional mandate. At the subnational level, UNDP will engage with the new 
Government to discuss policy options, including on fiscal decentralization and 
accountability mechanisms, for improved participation and inclusive decisionmaking, including by acknowledging the specific role of traditional governance 
systems such as Shuras and Jirgas in the Afghan context. UNDP is also planning to 
strengthen the capacities of elected provincial councils, district coordination 
councils and municipal boards. This will further the institutionalization of 
democratic processes as well as ensure equitable economic development and 
promotion of justice, the rule of law and women’s rights as core priorities of UNDP 
development interventions at the subnational level.
(b) Institutional capacity for peace-building. Building on the Afghanistan 
Peace and Reintegration Programme and possible successor arrangements, UNDP 
will adopt a two-pronged approach to address the root causes of conflict: (i) At the 
"""

index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))
graph = index_creator.from_text(text)
triples = graph.get_triples()

# # Create a dictionary to represent the knowledge graph
# graph_data = {
#     "nodes": list(set(triplet[0] for triplet in triples + [triplet[1] for triplet in triples])),
#     "edges": [{"source": triplet[0], "target": triplet[1], "label": triplet[2]} for triplet in triples]
# }

# # Convert graph data to JSON format
# graph_json = json.dumps(graph_data, indent=4)

# # Display or use the JSON data as needed
# print(graph_json)


### Save Graph 


In [13]:
graph.write_to_gml("moonshot3.gml")

### Use Saved Graph


In [20]:
from langchain.indexes.graph import NetworkxEntityGraph
loaded_graph = NetworkxEntityGraph.from_gml("combined_graph.gml")

question = f"""
what benefits does UNDP offer to Afghanistan
"""
chain = GraphQAChain.from_llm(OpenAI(temperature=0), graph=loaded_graph, verbose=True)
chain.run(question) 



> Entering new GraphQAChain chain...
Entities Extracted:
 UNDP, Afghanistan
Full Context:
UNDP will continue to adapt to a changing national context
UNDP will take on leadership or facilitation roles in those areas as requested
UNDP will apply conflict sensitivity
UNDP will ensure four cross-cutting parameters or guiding principles are addressed
UNDP will promote inclusive political processes and representative institutions
UNDP will support the National Assembly
UNDP will engage with the new Government
UNDP will discuss policy options including on fiscal decentralization and accountability mechanisms
UNDP will adopt a two-pronged approach to address the root causes of conflict
Afghanistan has presidential elections
Afghanistan has withdrawal of international troops
Afghanistan has closure of provincial reconstruction teams
Afghanistan has full sovereignty
Afghanistan has close to 9,000 civilian casualties
Afghanistan has Millennium Development Goals for education and health
Afghanis

' UNDP offers a range of benefits to Afghanistan, including leadership or facilitation roles in areas requested, conflict sensitivity, promotion of inclusive political processes and representative institutions, support for the National Assembly, engagement with the new Government, discussion of policy options including on fiscal decentralization and accountability mechanisms, and a two-pronged approach to address the root causes of conflict.'

### Merge Saved Files

In [16]:
import networkx as nx

# Read the first GML file
G1 = nx.read_gml('moonshot.gml')

# Read the second GML file
G2 = nx.read_gml('moonshot2.gml')

# Read the third GML file
G3 = nx.read_gml('moonshot3.gml')

# Read the third GML file
G4 = nx.read_gml('combined_graph.gml')


# Combine the graphs
combined_graph = nx.compose(G3, G4)

# Write the combined graph to a new GML file
nx.write_gml(combined_graph, 'combined_graph.gml')
